<a href="https://colab.research.google.com/github/woraphonp-038-5/Project_LineChatbot_Contraception/blob/main/%E0%B8%96%E0%B8%AD%E0%B8%94%E0%B8%84%E0%B8%A7%E0%B8%B2%E0%B8%A1_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ถอดความ**

In [ ]:
!pip install pdf2docx pymupdf pdfplumber pytesseract pdf2image python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.6 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:14 http://s

In [ ]:
# =========================
# Import libraries ที่จำเป็น
# =========================
import fitz                     # PyMuPDF ใช้ตรวจสอบและดึงข้อความจาก PDF (เช็กว่าเป็น text-based หรือไม่)
import pdfplumber               # ใช้ดึงตาราง (tables) จากไฟล์ PDF
from pdf2docx import Converter  # ใช้แปลง PDF → Word โดยพยายามรักษา layout เดิม
from docx import Document       # ใช้สร้างและแก้ไขไฟล์ Word (.docx)
from pdf2image import convert_from_path  # แปลงหน้า PDF เป็นรูปภาพ (ใช้ตอน OCR)
import pytesseract              # OCR: แปลงภาพเป็นข้อความ
import re                       # ใช้จัดการ regex สำหรับลบอักขระต้องห้ามใน XML

# ==========================================================
# กำหนดค่าพื้นฐานของไฟล์และช่วงหน้าที่ต้องการแปลง
# ==========================================================
PDF_FILE = "/content/WHO SPR 2025 4th edition .pdf"   # ไฟล์ PDF ต้นทาง
OUTPUT_DOCX = "WHO SPR 2025 4th edition.docx"        # ไฟล์ Word ปลายทาง

START_PAGE = 1     # หน้าเริ่มต้น (นับจาก 1)
END_PAGE = 108     # หน้าสุดท้าย

# ==========================================================
# regex สำหรับลบอักขระที่ Word (XML) ไม่รองรับ
# ป้องกัน error ตอนบันทึกไฟล์ .docx
# ==========================================================
ILLEGAL_XML_CHARS_PATTERN = re.compile(
    r"[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]"
)

def clean_xml_text(text):
    """
    ลบอักขระต้องห้ามใน XML ออกจากข้อความ
    เพื่อป้องกันไฟล์ Word เปิดไม่ได้
    """
    return ILLEGAL_XML_CHARS_PATTERN.sub("", text or "")

# ตรวจสอบว่า PDF เป็น text-based หรือ scanned
# ถ้ามีข้อความให้ดึงได้ → ถือว่าเป็น text-based
def has_text(pdf_path, start, end):
    with fitz.open(pdf_path) as doc:
        # วนดูแต่ละหน้าตามช่วงที่กำหนด
        for i in range(start - 1, min(end, len(doc))):
            # ถ้าหน้าใดมีข้อความ (ไม่ใช่ค่าว่าง)
            if doc[i].get_text().strip():
                return True
    return False

# แปลง PDF → Word ด้วย pdf2docx
# เหมาะสำหรับ PDF ที่เป็น text-based
def convert_with_pdf2docx(pdf_path, docx_path, start, end):
    cv = Converter(pdf_path)
    cv.convert(
        docx_path,
        start=start - 1,   # pdf2docx ใช้ index หน้าเริ่มจาก 0
        end=end            # หน้าสุดท้าย
    )
    cv.close()

# ดึงเฉพาะ "ตาราง" จาก PDF ด้วย pdfplumber
# ตารางจะถูกแปลงเป็น text แบบคั่นด้วย tab
def extract_tables(pdf_path, start, end):
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(start - 1, min(end, len(pdf.pages))):
            # ดึงทุกตารางในแต่ละหน้า
            for table in pdf.pages[i].extract_tables():
                for row in table:
                    # รวม cell แต่ละช่องในแถวเดียวกันด้วย tab
                    rows.append("\t".join(cell or "" for cell in row))
                rows.append("")  # เว้นบรรทัดระหว่างตาราง
    return clean_xml_text("\n".join(rows))

# OCR: ใช้เมื่อ PDF เป็น scanned (ไม่มี text ให้ดึง)
# แปลง PDF → image → text
def extract_text_ocr(pdf_path, start, end):
    texts = []

    # แปลงหน้า PDF เป็นภาพ
    images = convert_from_path(
        pdf_path,
        dpi=200,
        first_page=start,
        last_page=end
    )

    # OCR ทีละภาพ
    for img in images:
        text = pytesseract.image_to_string(img, lang="eng")
        texts.append(clean_xml_text(text))

    return "\n".join(texts)

# MAIN PROGRAM
if has_text(PDF_FILE, START_PAGE, END_PAGE):
    # -------------------------------
    # กรณี PDF เป็น text-based
    # -------------------------------
    print("Text-based PDF")

    # แปลง PDF เป็น Word โดยรักษา layout
    convert_with_pdf2docx(
        PDF_FILE,
        OUTPUT_DOCX,
        START_PAGE,
        END_PAGE
    )

    # ดึงตารางทั้งหมดจาก PDF
    tables = extract_tables(PDF_FILE, START_PAGE, END_PAGE)

    # ถ้ามีตาราง
    if tables.strip():
        doc = Document(OUTPUT_DOCX)
        doc.add_page_break()
        doc.add_heading("Extracted Tables", level=1)

        # เพิ่มตารางในรูปแบบข้อความ
        for line in tables.split("\n"):
            doc.add_paragraph(line)

        doc.save(OUTPUT_DOCX)

else:
    # -------------------------------
    # กรณี PDF เป็น scanned → ใช้ OCR
    # -------------------------------
    print("Scanned PDF → OCR")

    doc = Document()
    doc.add_heading("OCR Result", level=1)

    # เพิ่มข้อความ OCR ลงใน Word
    for para in extract_text_ocr(PDF_FILE, START_PAGE, END_PAGE).split("\n\n"):
        doc.add_paragraph(para)

    doc.save(OUTPUT_DOCX)

print("Finished:", OUTPUT_DOCX)


Text-based PDF
Finished: WHO SPR 2025 4th edition.docx
